In [10]:
import requests
import pandas as pd
import json
from dateutil import parser

In [11]:
API_KEY = "c03c82ec80c727e2730e5c0dcdb0ee48-30315c7d749cba7775f4afc24864fee3"
ACCOUNT_ID = "101-004-25825291-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [12]:
session = requests.Session()

In [13]:
 session.headers.update({
     "Authorization": f"Bearer {API_KEY}",
     "content-type": "application/json" 
 })

In [14]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [15]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [16]:
response = session.get(url, params=None, data=None, headers=None)

In [17]:
response.status_code

200

In [18]:
data = response.json()

In [19]:
instruments_list = data['instruments']

In [20]:
len(instruments_list)

124

In [21]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [22]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [23]:
""" Création d'un dictionnaire qui va avoir pour clé
les noms des devises du forex (instruments) et pour valeur
un autre dictionnaire qui a pour clé k venant de liste key_i
et pour valeur, sa valeur dans l'instrument. 
Le principe est de réduire la taille du dictionnaire instrument
qui est renvoyé par l'API et ne garder que le nécessaire"""
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }

In [24]:
instruments_dict

{'NL25_EUR': {'name': 'NL25_EUR',
  'type': 'CFD',
  'displayName': 'Netherlands 25',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.10'},
 'CHINAH_HKD': {'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'EUR_ZAR': {'name': 'EUR_ZAR',
  'type': 'CURRENCY',
  'displayName': 'EUR/ZAR',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'EUR_CHF': {'name': 'EUR_CHF',
  'type': 'CURRENCY',
  'displayName': 'EUR/CHF',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.04'},
 'NAS100_USD': {'name': 'NAS100_USD',
  'type': 'CFD',
  'displayName': 'US Nas 100',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.05'},
 'USD_PLN': {'name': 'USD_PLN',
  'type': 'CURRENCY',
  'displayName': 'USD/PLN',


In [25]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [26]:
"""Fonction de récupération des chandeliers
On définit la paire de devises, le nombre de chandeliers et la périodicité du chandelier"""
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

"""Fonction de création d'un dataframe à partir des données précédentes
On utilise les données précédentes pour passer de la forme d'un dictionnaire json
à un tableau de valeurs plus compréhensible."""
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame.empty
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] =  float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

"""Création d'un fichier à partir du dataframe précédemment créé."""
def create_data_file(pair_name, count=20, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failded !", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} {candles_df.time.min()} {candles_df.time.max()}")

In [ ]:
# code, data =  fetch_candles("EUR_USD", count=20, granularity="H4")
# candles_df = get_candles_df(data)

In [ ]:
# create_data_file("EUR_USD", count=10, granularity="H4")

In [27]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']


In [ ]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 2022-12-11 22:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_USD H4 4000 2021-01-10 22:00:00+00:00 2023-08-02 09:00:00+00:00
EUR_GBP H1 4000 2022-12-11 22:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_GBP H4 4000 2021-01-10 22:00:00+00:00 2023-08-02 09:00:00+00:00
EUR_JPY H1 4000 2022-12-11 22:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_JPY H4 4000 2021-01-08 14:00:00+00:00 2023-08-02 09:00:00+00:00
EUR_CHF H1 4000 2022-12-11 22:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_CHF H4 4000 2021-01-10 22:00:00+00:00 2023-08-02 09:00:00+00:00
EUR_NZD H1 4000 2022-12-09 21:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_NZD H4 4000 2021-01-10 22:00:00+00:00 2023-08-02 09:00:00+00:00
EUR_CAD H1 4000 2022-12-11 22:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_CAD H4 4000 2021-01-10 22:00:00+00:00 2023-08-02 09:00:00+00:00
EUR_AUD H1 4000 2022-12-09 21:00:00+00:00 2023-08-02 13:00:00+00:00
EUR_AUD H4 4000 2021-01-10 22:00:00+00:00 2023-08-02 09:00:00+00:00
USD_JPY H1 4000 2022-12-11 22:00:00+00:00 2023-0